In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import io
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
from zipfile import ZipFile
from statsmodels.tsa.seasonal import STL
from causalimpact import CausalImpact

In [70]:
def_colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
period_in_days = 7
fcn_compare = lambda a,b: abs(a-b)/max(a,b)


# Season-Trend decomposition using LOESS.
def decompose_signal(input_signal:pd.Series, period_in_days=14, minimum_heartbeat=0.85) -> pd.DataFrame:
    sales_decomposition_LOESS = STL(input_signal, period=period_in_days).fit()
    seasonality_flag = sales_decomposition_LOESS.trend > minimum_heartbeat
    df = pd.DataFrame({
        'heartbeat_flag': seasonality_flag,
        'trend': sales_decomposition_LOESS.trend,
        'seasonal': sales_decomposition_LOESS.seasonal,
        'residual': sales_decomposition_LOESS.resid
    })
    return df


def compare_promo_regular_sales(sales:pd.Series, promo:pd.Series, inferred_availability:pd.Series, idx_holiday_to_exclude:pd.Series, min_promo_days=3, min_regular_days=6) -> dict:
    '''
        Explain this well as it will go on the paper...
        
        This method splits a CFAV SKU into promotional and regular chunks taking into account the inferred availability (using LOESS)\
        and the holiday periods (any other event can be included in that idx).
        
        The method divides the selling days into sequences of regular and normal days and calculates marginal sales.
    
        'avg_promo_sales' and 'avg_regular_sales' are the sales aggregated across all the slots, whereas
        'slot_promo_avg_sales' and 'slot_promo_avg_sales' represent each slot.
        
        The total sales and total days are not returned, will I need them?
        
        min_promo_days=3, min_regular_days=6 decide the minimum number of days to be taken into consideration for the sequences.
        
            TO-DO: Add the beginning and end of the promotional periods
        
        Updates:
        25.10.2020 - First attempt
    
    '''

    analysis_results = []
    
    # only if there are promos
    if promo.sum() > 0:

        availability_sku_A = inferred_availability & (~idx_holiday_to_exclude)
        availability_value_sku_A = availability_sku_A.sum()/len(availability_sku_A)

        # Split the promotions into slots
        idx_pre_intervention, idx_post_intervention = split_promos_into_sequences(promo, min_promo_days=min_promo_days, min_regular_days=min_regular_days)

        num_promo_slots = len(idx_pre_intervention)

        slot_promo_sales = np.zeros(num_promo_slots)
        slot_regular_sales = np.zeros(num_promo_slots)

        slot_promo_days = np.zeros(num_promo_slots)
        slot_regular_days = np.zeros(num_promo_slots)

        for idx_promo_slot in range(0, num_promo_slots):
            idx_pre_intervention_current = idx_pre_intervention[idx_promo_slot]
            idx_post_intervention_current = idx_post_intervention[idx_promo_slot]

            slot_promo_sales[idx_promo_slot] = sales[idx_post_intervention_current].sum()
            slot_promo_days[idx_promo_slot] = idx_post_intervention_current.sum()

            slot_regular_sales[idx_promo_slot] = sales[idx_pre_intervention_current].sum()
            slot_regular_days[idx_promo_slot] = idx_pre_intervention_current.sum()

        slot_promo_avg_sales = np.divide(slot_promo_sales, slot_promo_days)
        slot_regular_avg_sales = np.divide(slot_regular_sales, slot_regular_days)
        # totals
        total_slot_promo_days = slot_promo_days.sum()
        if total_slot_promo_days>0:
            avg_promo_sales = slot_promo_sales.sum()/total_slot_promo_days
        else:
            avg_promo_sales = 0
        
        total_slot_regular_days = slot_regular_days.sum()
        if total_slot_regular_days>0:
            avg_regular_sales = slot_regular_sales.sum()/total_slot_regular_days
        else:
            avg_regular_sales = 0
            

        # difference between the averages during promo and regular
        difference_averages_promo_to_regular = avg_promo_sales-avg_regular_sales
        # cumulative difference
        cum_difference_sales_promo_to_regular = slot_promo_sales.sum()-slot_regular_sales.sum()
        
        analysis_results.append({
            'num_promo_slots': num_promo_slots,
            'avg_promo_sales': avg_promo_sales,
            'avg_regular_sales': avg_regular_sales,
            'promo_days': total_slot_promo_days, 
            'regular_days':total_slot_regular_days,
            'difference_averages_promo_to_regular': difference_averages_promo_to_regular,
            'cum_difference_sales_promo_to_regular': cum_difference_sales_promo_to_regular,
            'slot_promo_avg_sales': slot_promo_avg_sales,
            'slot_regular_avg_sales': slot_regular_avg_sales,
            'availability_value_sku_A': availability_value_sku_A
        })
    
    return analysis_results

def holiday_to_exclude(df_summary:pd.DataFrame, holidays):
    idx_holidays  = df_summary.index.isin(holidays)
    idx_covid19 = (df_summary.index >= 20210701) & (df_summary.index <= 20211001)
    return idx_holidays | idx_covid19

def customers_by_date_shop_pairs(customers_history_AB:'pd.DataFrame', period_in_days=7):
    df = customers_history_AB\
        .groupby(['date_key','shop_code'], as_index=False)\
        .agg(
            customers=('customer_key','nunique')
        )\
        .pivot(index='date_key',columns='shop_code',values='customers')\
        .fillna(0)\
        .applymap(int)
    return df

def customers_cross_from_victim_shop(customers_history_AB:'pd.DataFrame'):

    shop_code_A, shop_code_B = customers_history_AB.shop_code.unique()

    df = customers_history_AB\
        .assign(
            is_cross_customer = lambda s: (s.date_key > s.groupby('customer_key')['date_key'].transform('min')) & (s.date_key == s.groupby(['customer_key','shop_key'])['date_key'].transform('min'))
        )\
        .query(f"is_cross_customer")\
        .groupby(['date_key','shop_code'], as_index=False)\
        .agg(
            cross_customers=('customer_key','nunique')
        )\
        .pivot(index='date_key', columns='shop_code', values='cross_customers')\
        .rename(columns={shop_code_A:f'{shop_code_A}_cross',shop_code_B:f'{shop_code_B}_cross'})\
        .fillna(0)\
        .applymap(int)
    
    if f'{shop_code_A}_cross' not in df.columns:
        df[f'{shop_code_A}_cross'] = 0
    
    if f'{shop_code_B}_cross' not in df.columns:
        df[f'{shop_code_B}_cross'] = 0
    
    return df

# def customers_component_AB(decompostition):
#     shop_code_A, shop_code_B = [col for col in customers_by_date_shop_pairs.columns if isinstance(col, int)]
#     customers_shop_A = customers_by_date_shop_pairs[shop_code_A]
#     df_decomposition_A = decompose_signal()



# def compare_cross_regular_customers(df_store:'pd.DataFrame', df_history:'pd.DataFrame', df_components:'pd.DataFrame', shop_code_A, shop_code_B, min_date_key, holidays, min_cross_days=5, min_regular_days=10):

#     df_cross = df_history\
#         .query(f'shop_code.isin({[shop_code_A,shop_code_B]})')\
#         .assign(
#             is_cross_customer = lambda s: (s.date_key > s.groupby('customer_key')['date_key'].transform('min')) & (s.date_key == s.groupby(['customer_key','shop_key'])['date_key'].transform('min'))
#         )\
#         .query(f"is_cross_customer")\
#         .groupby(['date_key','shop_code'], as_index=False)\
#         .agg(
#             cross_customers=('customer_key','nunique')
#         )\
#         .pivot(index='date_key', columns='shop_code', values='cross_customers')\
#         .reset_index()\
#         .rename(columns={shop_code_A:f'{shop_code_A}_cross',shop_code_B:f'{shop_code_B}_cross'})\
#         .fillna(0)\
#         .applymap(int)
    
#     df_summary = df_store[['date_key',shop_code_A,shop_code_B]]\
#         .merge(df_cross, on='date_key', how='left')\
#         .fillna(0)\
#         .applymap(int)\
#         .query(f'date_key >= {min_date_key}')
#     df_summary[[f'{shop_code_A}_iscross',f'{shop_code_B}_iscross']] = df_summary[[f'{shop_code_A}_cross',f'{shop_code_B}_cross']].applymap(lambda x: False if x==0 else True)

#     customers_shop_A = df_summary[shop_code_A]
#     cross_shop_A = df_summary[f'{shop_code_A}_iscross']
#     inferred_availability_shop_A = df_components.query(f'date_key >= {min_date_key}')[f'{shop_code_A}_heartbeat_flag']
    
#     analysis_results = []
#     idx_holiday_to_exclude = holiday_to_exclude(df_summary, holidays)
    
#     # only if there are cross shops
#     if cross_shop_A.sum() > 0:

#         availability_shop_A = inferred_availability_shop_A & (~idx_holiday_to_exclude)
#         availability_value_shop_A = availability_shop_A.sum()/len(availability_shop_A)

#         # Split the cross shops into slots
#         idx_pre_intervention, idx_post_intervention = split_cross_into_sequences(cross_shop_A, \
#             min_cross_days=min_cross_days, min_regular_days=min_regular_days)

#         num_cross_slots = len(idx_pre_intervention)

#         slot_cross_customers = np.zeros(num_cross_slots)
#         slot_regular_customers = np.zeros(num_cross_slots)

#         slot_cross_days = np.zeros(num_cross_slots)
#         slot_regular_days = np.zeros(num_cross_slots)

#         for idx_cross_slot in range(0, num_cross_slots):
#             idx_pre_intervention_current = idx_pre_intervention[idx_cross_slot]
#             idx_post_intervention_current = idx_post_intervention[idx_cross_slot]

#             slot_cross_customers[idx_cross_slot] = customers_shop_A[idx_post_intervention_current].sum()
#             slot_cross_days[idx_cross_slot] = idx_post_intervention_current.sum()

#             slot_regular_customers[idx_cross_slot] = customers_shop_A[idx_pre_intervention_current].sum()
#             slot_regular_days[idx_cross_slot] = idx_pre_intervention_current.sum()

#         slot_cross_avg_customers = np.divide(slot_cross_customers, slot_cross_days)
#         slot_regular_avg_customers = np.divide(slot_regular_customers, slot_regular_days)
#         # totals
#         total_slot_cross_days = slot_cross_days.sum()
#         if total_slot_cross_days>0:
#             avg_cross_customers = slot_cross_customers.sum()/total_slot_cross_days
#         else:
#             avg_cross_customers = 0
        
#         total_slot_regular_days = slot_regular_days.sum()
#         if total_slot_regular_days>0:
#             avg_regular_customers = slot_regular_customers.sum()/total_slot_regular_days
#         else:
#             avg_regular_customers = 0
            

#         # difference between the averages during cross shops and regular
#         difference_averages_cross_to_regular = avg_cross_customers-avg_regular_customers
#         # cumulative difference
#         cum_difference_customers_cross_to_regular = slot_cross_customers.sum()-slot_regular_customers.sum()
        
#         analysis_results.append({
#             'shop_code_A':shop_code_A,
#             'shop_code_B':shop_code_B,
#             'num_cross_slots': num_cross_slots,
#             'avg_cross_customers': avg_cross_customers,
#             'avg_regular_customers': avg_regular_customers,
#             'cross_days': total_slot_cross_days, 
#             'regular_days':total_slot_regular_days,
#             'difference_averages_cross_to_regular': difference_averages_cross_to_regular,
#             'cum_difference_customers_cross_to_regular': cum_difference_customers_cross_to_regular,
#             'slot_promo_avg_customers': slot_cross_avg_customers,
#             'slot_regular_avg_customers': slot_regular_avg_customers,
#             'availability_value_shop_A': availability_value_shop_A
#         })
    
#     return analysis_results


def calculate_causal_impact_with_covariates(
        promo_sku_A:pd.Series, 
        availability_sku_A:pd.Series,
        sales_sku_B:pd.Series,
        promo_sku_B:pd.Series, 
        availability_sku_B:pd.Series,
        idx_pre_intervention:pd.Series, 
        idx_post_intervention:pd.Series,
        idx_holiday_to_exclude:pd.Series,
        min_diff_in_units_from_reg_to_promo, 
        min_ratio_change = 0.3,
        do_exclude_promos_SKU_B = True, 
        be_verbose=True,
        min_overlapping_days_regular=10,
        min_overlapping_days_promo=5
    ):
    '''
        This is the method used to populate the paper results
    '''

    # use this flag to exclude sku_B if on promo
    total_days = promo_sku_A.shape[0]
    num_days = promo_sku_A.index
    combined_availability = availability_sku_A & availability_sku_B & (~idx_holiday_to_exclude)

    causal_analysis = []

    # sales_sku_B = df_sales_covariates.iloc[:,0]

    total_slots = len(idx_pre_intervention)

    for idx_promo_slot in range(0, total_slots):

        idx_pre_intervention_current = idx_pre_intervention[idx_promo_slot]
        idx_post_intervention_current = idx_post_intervention[idx_promo_slot]

        # # #
        # promo days == 'post-intervention'
        # # #
        idx_overlapping_days_promo = combined_availability & idx_post_intervention_current
        total_overlapping_days_promo = idx_overlapping_days_promo.sum()


        # overlapping promo days. Both SKUs on promo, "competing promos"
        idx_competing_promo_days = idx_overlapping_days_promo & promo_sku_B
        competing_promo_days = idx_competing_promo_days.sum()


        # # #
        # regular days == 'pre-intervention'
        # # #
        # A period should not be marked as 'regular' if SKU_B is on promotion
        if do_exclude_promos_SKU_B:
            idx_overlapping_days_regular = combined_availability & idx_pre_intervention_current & (~promo_sku_B)
        else:
            idx_overlapping_days_regular = combined_availability & idx_pre_intervention_current
        total_overlapping_days_regular = idx_overlapping_days_regular.sum()
        
        # Minimum requirements of overlap. Otherwise the analysis does not make much sense.
        # numerical index (for Causal Impact)
        if (total_overlapping_days_regular>=min_overlapping_days_regular) & (total_overlapping_days_promo>=min_overlapping_days_promo):
            
            ind_regular_days = num_days[idx_overlapping_days_regular]
            start_regular = ind_regular_days.min()
            end_regular = ind_regular_days.max()
            
            ind_promo_days = num_days[idx_overlapping_days_promo]
            start_promo = ind_promo_days.min()
            end_promo = ind_promo_days.max()
            gap_days = (start_promo - end_regular).days - 1

            # sales of SKU_B
            # during regular
            sku_B_regular_avg_sales = sales_sku_B[idx_overlapping_days_regular].mean()
            # during promo
            sku_B_avg_sales_during_promo_sku_A = sales_sku_B[idx_overlapping_days_promo].mean()
            # Difference in average sales between the regular and the promotional one.
            diff_in_units_from_reg_to_promo = sku_B_regular_avg_sales-sku_B_avg_sales_during_promo_sku_A
            
            # can we review the post promotional sales?
            end_promo_loc = num_days.get_loc(end_promo)
            post_period_start_loc = end_promo_loc+1
            post_period_end_loc = min(end_promo_loc+1+min_overlapping_days_regular, total_days)
            sku_B_regular_post_promo = sales_sku_B.iloc[post_period_start_loc:post_period_end_loc]
            post_promo_days = sku_B_regular_post_promo.shape[0]
            sku_B_regular_post_promo_avg_sales = sku_B_regular_post_promo.mean()
            # post promo should be larger than during the cannibalisation
            diff_in_units_from_promo_to_pos_promo = sku_B_avg_sales_during_promo_sku_A-sku_B_regular_post_promo_avg_sales
            
            post_promo_flag = (-diff_in_units_from_promo_to_pos_promo > min_diff_in_units_from_reg_to_promo*0.25)
            '''
            if idx_promo_slot+1 < total_slots:
                # during regular
                idx_reg_post_intervention = idx_pre_intervention[idx_promo_slot+1]
                sku_B_regular_post_promo_avg_sales = sales_sku_B[idx_reg_post_intervention].mean()
                # post promo should be larger than during the cannibalisation
                diff_in_units_from_promo_to_pos_promo = sku_B_avg_sales_during_promo_sku_A-sku_B_regular_post_promo_avg_sales
                post_promo_flag = (-diff_in_units_from_promo_to_pos_promo>min_diff_in_units_from_reg_to_promo)
            else:
                diff_in_units_from_promo_to_pos_promo = np.nan
                post_promo_flag = True
            '''

            # delta
            ratio_change = fcn_compare(sku_B_avg_sales_during_promo_sku_A, sku_B_regular_avg_sales)
            if be_verbose:
                print(f'Summary of the current scenario (slot {idx_promo_slot})')
                print(f'Before SKU A going on promo, the SKUs overlap for {total_overlapping_days_regular} days (promos on sku_B excluded: {do_exclude_promos_SKU_B})')
                
                if gap_days > 0:
                    print(f'There is a gap of {gap_days} days between the regular days and the beginning of the promotion (due to availability/promotional period of SKU_B)')
                print(f'When SKU_A is on promo, the SKUs overlap for {total_overlapping_days_promo} days')
                print(f'During the overlapping days, SKU B is on promo for {competing_promo_days} days')

                print(f'Average sales of sku B before sku A on promo {sku_B_regular_avg_sales:.2f}')
                print(f'Average sales of sku B during sku A on promo {sku_B_avg_sales_during_promo_sku_A:.2f}')
                print(f'Average sales of sku B after sku A on promo {sku_B_regular_post_promo_avg_sales:.2f} over {post_promo_days} days - {post_promo_flag}')
                print(f'Diff in units from regular to promotion {-diff_in_units_from_reg_to_promo:.2f}')
                print(f'Diff in units from cannibalisation to regular {-diff_in_units_from_promo_to_pos_promo:.2f}')
                
                print(f'Ratio of change {ratio_change:3.2f} (the lower the closer (0,1)) {ratio_change>min_ratio_change} \n')


            if (ratio_change>min_ratio_change) & (diff_in_units_from_reg_to_promo > min_diff_in_units_from_reg_to_promo) & post_promo_flag:

                idx_regular_days = np.array([start_regular, end_regular]).tolist()
                idx_promo_days   = np.array([start_promo, end_promo]).tolist()

                print('Running Causal Impact...')
                ci = CausalImpact(sales_sku_B, idx_regular_days, idx_promo_days)
                '''
                https://github.com/dafiti/causalimpact/blob/8d881fc5c270348d8c8ff59c936997a75d7c5fac/causalimpact/main.py#L88
                
                First column must contain the `y` measured value 
                while the others contain the covariates `X` that are used in the 
                linear regression component of the model.
                '''
                #ci.lower_upper_percentile
                avg_actual = ci.summary_data.loc['actual', 'average']
                # Had the promo not been launched on the cannibal, we would have sold this amount.
                avg_predicted = ci.summary_data.loc['predicted', 'average']
                avg_abs_effect = ci.summary_data.loc['abs_effect', 'average']
                # This can be seen as the number of units that the cannibal is taking from the victim
                cum_abs_effect = ci.summary_data.loc['abs_effect', 'cumulative']
                posterior_tail_prob = ci.p_value
                prob_causal_effect = (1-ci.p_value)*100
                print(f'CausalImpact >> Probability of a causal event {prob_causal_effect:.2f}')

                temp_dict = {
                    'slot_number': idx_promo_slot,
                    'idx_regular_days': idx_regular_days,
                    'idx_promo_days': idx_promo_days,
                    'total_overlapping_days_regular': total_overlapping_days_regular,
                    'regular_to_promo_gap': gap_days,
                    'total_overlapping_days_promo': total_overlapping_days_promo,
                    'competing_promo_days': competing_promo_days,
                    'sku_B_regular_avg_sales': sku_B_regular_avg_sales,
                    'sku_B_avg_sales_during_promo_sku_A': sku_B_avg_sales_during_promo_sku_A,
                    'diff_in_units_from_reg_to_promo': diff_in_units_from_reg_to_promo,
                    'diff_in_units_from_promo_to_pos_promo': diff_in_units_from_promo_to_pos_promo,
                    'ratio_change': ratio_change,
                    'avg_actual':avg_actual,
                    'avg_predicted': avg_predicted,
                    'avg_abs_effect': avg_abs_effect,
                    'cum_abs_effect': cum_abs_effect,
                    'posterior_tail_prob': posterior_tail_prob,
                    'prob_causal_effect': prob_causal_effect
                }
                causal_analysis.append(temp_dict)
                
    return causal_analysis

def add_graph_relationship(node_A, node_B, edge_properties: dict):
    DG = nx.DiGraph()

    DG.add_node(node_A['name'], **node_A['properties'])

    d = dict()
    DG.add_node(node_B['name'], **node_B['properties'])

    edge_label = '\n'.join([f'{k}: {v:3.2f}' for k,v in edge_properties.items()])  
    DG.add_edge(node_A['name'], node_B['name'], **edge_properties, label=edge_label)

        
def split_promos_into_sequences(idx_promos:pd.Series, min_promo_days=5, min_regular_days=10) -> tuple:
    '''
    Group the indices of a promotion into sequences of pre and post promotion
    '''
    # Groups/sequences
    seqs = (idx_promos.shift(1)!=idx_promos).cumsum()
    promo_seqs = seqs[idx_promos]
    # Indices
    idx_pre_intervention = []
    idx_post_intervention = []
    for value_promo_seqs in promo_seqs.unique():
        idx_current_promo = seqs==value_promo_seqs
        prev_seq = value_promo_seqs-1
        idx_current_regular = seqs==prev_seq
        current_promo_length = idx_current_promo.sum()
        current_regular_length = idx_current_regular.sum()
        if (current_promo_length >= min_promo_days) and (current_regular_length >= min_regular_days):
            idx_pre_intervention.append(idx_current_regular)
            idx_post_intervention.append(idx_current_promo)
    return idx_pre_intervention, idx_post_intervention

def split_cross_into_sequences(idx_cross: 'pd.Series', min_cross_days=5, min_regular_days=10):
    # Groups/sequences
    seqs = (idx_cross.shift(1)!=idx_cross).cumsum()
    promo_seqs = seqs[idx_cross]
    # Indices
    idx_pre_intervention = []
    idx_post_intervention = []
    for value_promo_seqs in promo_seqs.unique():
        idx_current_promo = seqs==value_promo_seqs
        prev_seq = value_promo_seqs-1
        idx_current_regular = seqs==prev_seq
        current_promo_length = idx_current_promo.sum()
        current_regular_length = idx_current_regular.sum()
        if (current_promo_length >= min_cross_days) and (current_regular_length >= min_regular_days):
            idx_pre_intervention.append(idx_current_regular)
            idx_post_intervention.append(idx_current_promo)
    return idx_pre_intervention, idx_post_intervention


In [3]:
# read dimension data
product_hierarchy = pd.read_csv('./data/product_hierarchy.csv')
store_cities = pd.read_csv('./data/store_cities.csv')

# read fact data
sales_zip = ZipFile('./data/sales.csv.zip', 'r')
sales_data = sales_zip.read('sales.csv')
sales_bytes = io.BytesIO(sales_data)
sales_bytes.seek(0)
sales = pd.read_csv(sales_bytes, converters={'promo_bin_1':str,'promo_bin_2':str,'promo_discount_type_2':str})
df = sales.query("sales != 0 and ~sales.isnull()").reset_index(drop=True)
df["date"] = pd.to_datetime(df["date"])
df['promo'] = df[['promo_bin_1','promo_bin_2']].sum(axis=1) != ''

df_sales = df.reindex(columns=['product_id', 'store_id', 'date', 'revenue', 'promo'])

In [4]:
# import numpy as np
# import pandas as pd
# from statsmodels.tsa.arima_process import ArmaProcess
# from causalimpact import CausalImpact


# np.random.seed(12345)
# ar = np.r_[1, 0.9]
# ma = np.array([1])
# arma_process = ArmaProcess(ar, ma)
# X = 100 + arma_process.generate_sample(nsample=100)
# y = 1.2 * X + np.random.normal(size=100)
# y[70:] += 5

# data = pd.DataFrame({'y': y, 'X': X}, columns=['y', 'X'])
# pre_period = [0, 69]
# post_period = [70, 99]

# ci = CausalImpact(data, pre_period, post_period)
# print(ci.summary())
# print(ci.summary(output='report'))
# ci.plot()

In [4]:
# number of sales days by store

df_sales_daily = df_sales.groupby(["store_id","date"], as_index=False).agg(total_revenue=("revenue","sum"))

df_sales_daily.groupby("store_id", as_index=False).agg(
    sales_days=("date","count"),
    from_date=("date","min"),
    to_date=("date","max")
).sort_values("sales_days", ascending=False)

,store_id,sales_days,from_date,to_date
83,S0084,1033,2017-01-02,2019-10-31
62,S0063,1033,2017-01-02,2019-10-31
51,S0052,1033,2017-01-02,2019-10-31
26,S0027,1033,2017-01-02,2019-10-31
89,S0090,1033,2017-01-02,2019-10-31
...,...,...,...,...
135,S0136,312,2017-07-14,2019-09-08
16,S0017,304,2017-06-09,2019-09-08
6,S0007,213,2019-03-29,2019-10-31
56,S0057,171,2019-05-10,2019-10-31


it seems that the revenue of store S0084 dose not follow any trend like peaking during the weekends...

In [ ]:
# # use store S0084 as a sample
# df_sales_S0084 = df_sales_daily[(df_sales_daily["store_id"]=="S0084")&(df_sales_daily["date"].dt.year==2017)]
# df_sales_S0084_daily = df_sales_S0084.set_index("date")["total_revenue"]

# # Fig sizes
# fig_h = 10/1.5
# fig_w = 18/1.5

# # store_alias = 744
# f, ax = plt.subplots(1,1,figsize=(fig_w*1.5*1.5, fig_h*1.5))
# font_size = 13
# do_save_decomposition = True

# plt.rcParams.update({'font.size': font_size})
# def_colours = plt.rcParams['axes.prop_cycle'].by_key()['color']

# x_axis = df_sales_S0084_daily.index
# period_in_days=7



# decomposition_results = decompose_signal(df_sales_S0084_daily, period_in_days=period_in_days, minimum_heartbeat=0.85)
# df_decomposition = pd.DataFrame(decomposition_results)

# idx_store = 0


# idx_axis = 0
# '''ax.plot(x_axis, df_store.total_units, label=f'{dept_id} sales {store_alias}', 
#         color=def_colours[idx_store], linewidth=2, alpha=0.75)'''

# ax.plot(x_axis, df_sales_S0084_daily, label=f'Total revenue', 
#         color=def_colours[idx_store], linewidth=1+1, alpha=0.75)

# ax.plot(x_axis, df_decomposition['trend'], label=f'Trend', 
#         color=def_colours[idx_store+1], linewidth=1.5+1, alpha=0.95)

# ax.plot(x_axis, df_decomposition['seasonal'], label=f'Seasonal ({period_in_days} days)',
#         color=def_colours[idx_store+2], linewidth=1.5+1, alpha=0.95)

# ax.plot(x_axis, df_decomposition['residual'], label=f'Residual',
#         color=def_colours[idx_store+3], linewidth=0.5+1, alpha=0.65)


# plt.legend()
# plt.xlabel('date')
# plt.ylabel('revenue')
# plt.title("DAILY REVENUE OF STORE S0084")
# plt.grid(True)
# plt.tight_layout()
# plt.margins(0,0.05)


In [5]:
df_sales_pivot = df_sales.pivot(index=["store_id","date"], columns="product_id", values="revenue")
df_sales_pivot.fillna(0, inplace=True)
df_sales_pivot = df_sales_pivot.reset_index().merge(df_sales_daily)
df_sales_pivot['total_revenue_trend'] = df_sales_pivot.groupby("store_id")["total_revenue"].transform(lambda series: STL(series, period=period_in_days).fit().trend)
df_sales_pivot.set_index(["store_id","date"], inplace=True)

In [6]:
df_promotion_pivot = df_sales.pivot(index=["store_id","date"], columns="product_id", values="promo")
df_promotion_pivot.fillna(False, inplace=True)

product_list = list(df_promotion_pivot.columns)

In [7]:
df_heartbeat_flag = pd.DataFrame(index=df_sales_pivot.index)

# Season-Trend decomposition using LOESS
for product_A in tqdm(product_list):
    hearbeat_flag = df_sales_pivot.groupby(level=0)[product_A].transform(lambda series: decompose_signal(series, period_in_days=period_in_days, minimum_heartbeat=0.85)['heartbeat_flag'])
    df_heartbeat_flag = pd.concat([df_heartbeat_flag, hearbeat_flag], axis=1)

100%|██████████| 629/629 [08:27<00:00,  1.24it/s]


In [8]:
df_trend_pivot = pd.DataFrame(index=df_sales_pivot.index)

# Season-Trend decomposition using LOESS
for product_A in tqdm(product_list):
    trend = df_sales_pivot.groupby(level=0)[product_A].transform(lambda series: decompose_signal(series, period_in_days=period_in_days, minimum_heartbeat=0.85)['trend'])
    df_trend_pivot = pd.concat([df_trend_pivot, trend], axis=1)

100%|██████████| 629/629 [11:40<00:00,  1.11s/it]


In [9]:
df_residual_pivot = pd.DataFrame(index=df_sales_pivot.index)

# Season-Trend decomposition using LOESS
for product_A in tqdm(product_list):
    residual = df_sales_pivot.groupby(level=0)[product_A].transform(lambda series: decompose_signal(series, period_in_days=period_in_days, minimum_heartbeat=0.85)['residual'])
    df_residual_pivot = pd.concat([df_residual_pivot, residual], axis=1)

100%|██████████| 629/629 [12:00<00:00,  1.15s/it]


In [18]:
# List of products
sku_analysis = []

min_promo_days=5
min_regular_days=10
holidays = pd.Series([]).astype("datetime64[ns]") # assume that there is no holiday effect

for product_A in tqdm(product_list):

    temp1 = pd.concat([
            df_sales_pivot[[product_A]].rename(columns={product_A:"revenue"}), 
            df_promotion_pivot[[product_A]].rename(columns={product_A:"promo"}), 
            df_heartbeat_flag[[product_A]].rename(columns={product_A:"heartbeat"})
        ], axis=1)
    
    for store_id, temp2 in temp1.groupby(level=0):
        
        idx_holiday_to_exclude = temp2.index.get_level_values(1).isin(holidays)
        product_behaviour = compare_promo_regular_sales(
            sales=temp2["revenue"], 
            promo=temp2["promo"],
            inferred_availability=temp2["heartbeat"],
            idx_holiday_to_exclude=idx_holiday_to_exclude,
            min_promo_days=min_promo_days, 
            min_regular_days=min_regular_days
        )

        if product_behaviour != []:
            temp3 = product_behaviour[0]
            temp3["product_id"] = product_A
            temp3["store_id"] = store_id
            sku_analysis.append(temp3)
            
    
# Stick the dicts into a DF
df_snap_stats = pd.DataFrame(sku_analysis)
df_snap_stats.set_index(["store_id","product_id"], inplace=True)
# clean the empty dictionaries
idx_nonsense = df_snap_stats.num_promo_slots.isna() | (df_snap_stats.num_promo_slots < 1)
df_snap_stats = df_snap_stats[~idx_nonsense].copy()

# for backwards compatibility
df_snap_stats['mu_difference'] = df_snap_stats['difference_averages_promo_to_regular']
# Add a small offset to avoid 0-divisions
df_snap_stats['mu_delta'] = df_snap_stats.apply(lambda snap_reg: (snap_reg['avg_promo_sales']+0.01)/(snap_reg['avg_regular_sales']+0.01), axis=1)

# save the file
df_snap_stats.sort_values(by=['mu_delta'], ascending=False, inplace=True)

100%|██████████| 629/629 [01:17<00:00,  8.09it/s]


In [19]:
# Potential cannibals
# This threshold is used to detect uplifters based on the difference in average sales
sales_threshold = 1/3
# min AVG sales to be considered
min_avg_sales = 2

# A bit of work on the uplifters
# Snap sales greater than the sales + threshold
idx_A = df_snap_stats['mu_delta']>=(1+sales_threshold)

# Comparison only valid if they are not zero sales
idx_B = (df_snap_stats['avg_promo_sales']>min_avg_sales) & (df_snap_stats['avg_regular_sales']>min_avg_sales)
df_snap_stats['uplift_in_median'] = (idx_A & idx_B)

# Get two groups: potential cannibals (and haloers) and victims
df_snap_uplifters = df_snap_stats[df_snap_stats['uplift_in_median']].copy()

In [72]:
# If true, use the sales without the weekly pattern
do_decomposition = False

# This is the minimum bump between regular and promo
# and promo back to regular *0.25
min_diff_in_units_from_reg_to_promo = 2

# values for the CI analysis
min_ratio_change = 0.4
do_exclude_promos_SKU_B = True
# This flag is pretty good for debugging/development
be_verbose=False

# Option to deseasonalise
sku_potential_cannibals = df_snap_uplifters.index.tolist()

df_CI_analysis = pd.DataFrame()

total_cannibals = len(sku_potential_cannibals)

for idx, dimension in enumerate(sku_potential_cannibals):

    store_id, product_A = dimension
    print(f'{idx}/{total_cannibals}-{store_id}-{product_A}')

    df_promotion_sub = df_promotion_pivot[df_promotion_pivot.index.get_level_values(0)==store_id].reset_index(level=0)
    df_heartbeat_sub = df_heartbeat_flag[df_heartbeat_flag.index.get_level_values(0)==store_id].reset_index(level=0)
    df_sales_sub = df_sales_pivot[df_sales_pivot.index.get_level_values(0)==store_id].reset_index(level=0)
    df_trend_sub = df_trend_pivot[df_trend_pivot.index.get_level_values(0)==store_id].reset_index(level=0)
    df_residual_sub = df_residual_pivot[df_residual_pivot.index.get_level_values(0)==store_id].reset_index(level=0)
    idx_holiday_to_exclude = df_sales_sub.index.isin(holidays)

    # Get the promotions and split them into slots
    promo_sku_A = df_promotion_sub[product_A]
    idx_pre_intervention, idx_post_intervention = split_promos_into_sequences(promo_sku_A, min_promo_days=min_promo_days, min_regular_days=min_regular_days)

    availability_sku_A = df_heartbeat_sub[product_A]
    availability_value_sku_A = availability_sku_A.sum()/len(availability_sku_A)
    flag_min_availability_sku_A = availability_value_sku_A > 0.9

    # TO-DO: Decomposition should be done according to the SKU's patterns
    if flag_min_availability_sku_A & do_decomposition:
        sales_sku_A = df_trend_sub[product_A] + df_residual_sub[product_A]
    else:
        sales_sku_A = df_sales_sub[product_A]


    # go through all the SKUs in the store
    sku_potential_victims = product_list.copy()
    category_A = product_hierarchy[product_hierarchy["product_id"]==product_A]["hierarchy1_id"].iloc[0]
    sku_potential_victims = product_hierarchy[
        (product_hierarchy["hierarchy1_id"]==category_A)& # cannibalization should be only considered when products are in the same category
        (product_hierarchy["product_id"]!=product_A)& # product B must be different from product A
        (product_hierarchy["product_id"].isin(product_list)) # product B must have revenue
    ]["product_id"].tolist()

    for product_B in sku_potential_victims:
        availability_sku_B = df_heartbeat_sub[product_B]
        promo_sku_B = df_promotion_sub[product_B]

        # Decide what to do with sku_B on promo
        # - Remove the promo days? when? outside the cannibalisation window?
        # - We should compare windows of pre/post promo, not the entire year
        availability_value_sku_B = availability_sku_B.sum()/len(availability_sku_B)
        flag_min_availability_sku_B = availability_value_sku_B > 0.9

        if flag_min_availability_sku_B & do_decomposition:
            sales_sku_B = df_residual_sub[product_B] + df_trend_sub[product_B]
        else:
            sales_sku_B = df_sales_sub[product_B]
        
        ci_analysis = calculate_causal_impact_with_covariates(
                promo_sku_A=promo_sku_A,
                availability_sku_A=availability_sku_A,
                sales_sku_B=sales_sku_B, 
                promo_sku_B=promo_sku_B,
                availability_sku_B=availability_sku_B,
                idx_pre_intervention=idx_pre_intervention,
                idx_post_intervention=idx_post_intervention,
                idx_holiday_to_exclude=idx_holiday_to_exclude,
                min_diff_in_units_from_reg_to_promo=min_diff_in_units_from_reg_to_promo,
                min_ratio_change=min_ratio_change,
                do_exclude_promos_SKU_B=do_exclude_promos_SKU_B,
                be_verbose=be_verbose,
                min_overlapping_days_promo=min_promo_days,
                min_overlapping_days_regular=min_regular_days
            )
        if ci_analysis != []:
            print(f">>> {product_B}")
            causal_impact_analysis = pd.DataFrame(ci_analysis)
            causal_impact_analysis[["store_id","product_id_A","product_id_B"]] = store_id, product_A, product_B
            df_CI_analysis = pd.concat([df_CI_analysis, causal_impact_analysis])

0/1265-S0020-P0316
Running Causal Impact...
CausalImpact >> Probability of a causal event 92.31
>>> P0189
Running Causal Impact...
CausalImpact >> Probability of a causal event 83.92
>>> P0260
Running Causal Impact...
CausalImpact >> Probability of a causal event 96.20
>>> P0436
Running Causal Impact...
CausalImpact >> Probability of a causal event 95.10
>>> P0639
1/1265-S0051-P0305
Running Causal Impact...
CausalImpact >> Probability of a causal event 95.80
>>> P0428
2/1265-S0094-P0559
Running Causal Impact...
CausalImpact >> Probability of a causal event 73.03
>>> P0332
Running Causal Impact...
CausalImpact >> Probability of a causal event 99.00
>>> P0436
Running Causal Impact...
CausalImpact >> Probability of a causal event 87.61
>>> P0459
Running Causal Impact...
CausalImpact >> Probability of a causal event 95.50
>>> P0560
Running Causal Impact...
CausalImpact >> Probability of a causal event 95.20
>>> P0569
Running Causal Impact...
CausalImpact >> Probability of a causal event 99

: 